In [201]:
#https://github.com/tgel0/spotify-data/blob/master/notebooks/SpotifyDataRetrieval.ipynb
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="05a142b1de0f4d6795dce128df645791" 
secret = "da6c86eca5574ca1a6f666a8cfde0e35"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [202]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000,50):
    track_results = sp.search(q='year:2009 and genre:rap', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 8.894239249999998


In [203]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 1000


In [204]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Kid Cudi,Day 'N' Nite (nightmare),5FEXPoPnzueFJQCPRIrC3c,69
1,Kid Cudi,Day 'N' Nite (nightmare),5wfllwdjglyQislkskYLBv,69
2,Jay Sean,Down,6cmm1LMvZdB5zsCwX5BjqE,77
3,Kid Cudi,Up Up & Away,1RUTIdTnFs8lHSc0Zr4UJB,64
4,Drake,Forever,5UsLjwBaTHBX4ektWIr4XX,71


In [205]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  1000 non-null   object
 1   track_name   1000 non-null   object
 2   track_id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [206]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

96

In [207]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [208]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [209]:

df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [210]:
df_tracks.shape

(898, 4)

In [211]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.351437625000017


In [212]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 898


In [213]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (898, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.871,0.440,11,-6.250,0,0.0761,0.49900,0.000004,0.1140,0.799,137.996,audio_features,5FEXPoPnzueFJQCPRIrC3c,spotify:track:5FEXPoPnzueFJQCPRIrC3c,https://api.spotify.com/v1/tracks/5FEXPoPnzueF...,https://api.spotify.com/v1/audio-analysis/5FEX...,221267,4
1,0.727,0.680,2,-4.498,1,0.0286,0.01010,0.000000,0.0821,0.728,132.012,audio_features,6cmm1LMvZdB5zsCwX5BjqE,spotify:track:6cmm1LMvZdB5zsCwX5BjqE,https://api.spotify.com/v1/tracks/6cmm1LMvZdB5...,https://api.spotify.com/v1/audio-analysis/6cmm...,212507,4
2,0.660,0.881,10,-6.119,0,0.0849,0.07220,0.000034,0.3620,0.586,120.017,audio_features,1RUTIdTnFs8lHSc0Zr4UJB,spotify:track:1RUTIdTnFs8lHSc0Zr4UJB,https://api.spotify.com/v1/tracks/1RUTIdTnFs8l...,https://api.spotify.com/v1/audio-analysis/1RUT...,227347,4
3,0.457,0.906,5,-2.278,0,0.3420,0.24900,0.000000,0.1820,0.540,104.020,audio_features,5UsLjwBaTHBX4ektWIr4XX,spotify:track:5UsLjwBaTHBX4ektWIr4XX,https://api.spotify.com/v1/tracks/5UsLjwBaTHBX...,https://api.spotify.com/v1/audio-analysis/5UsL...,357707,5
4,0.382,0.996,7,-4.119,1,0.1040,0.00208,0.000000,0.4170,0.327,93.345,audio_features,2gscB6kDOmrv1P6qs2KXE3,spotify:track:2gscB6kDOmrv1P6qs2KXE3,https://api.spotify.com/v1/tracks/2gscB6kDOmrv...,https://api.spotify.com/v1/audio-analysis/2gsc...,147840,4


In [214]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      898 non-null    float64
 1   energy            898 non-null    float64
 2   key               898 non-null    int64  
 3   loudness          898 non-null    float64
 4   mode              898 non-null    int64  
 5   speechiness       898 non-null    float64
 6   acousticness      898 non-null    float64
 7   instrumentalness  898 non-null    float64
 8   liveness          898 non-null    float64
 9   valence           898 non-null    float64
 10  tempo             898 non-null    float64
 11  type              898 non-null    object 
 12  id                898 non-null    object 
 13  uri               898 non-null    object 
 14  track_href        898 non-null    object 
 15  analysis_url      898 non-null    object 
 16  duration_ms       898 non-null    int64  
 1

In [215]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(898, 14)

In [216]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (898, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Kid Cudi,Day 'N' Nite (nightmare),5FEXPoPnzueFJQCPRIrC3c,69,0.871,0.440,11,-6.250,0,0.0761,0.49900,0.000004,0.1140,0.799,137.996,221267,4
1,Jay Sean,Down,6cmm1LMvZdB5zsCwX5BjqE,77,0.727,0.680,2,-4.498,1,0.0286,0.01010,0.000000,0.0821,0.728,132.012,212507,4
2,Kid Cudi,Up Up & Away,1RUTIdTnFs8lHSc0Zr4UJB,64,0.660,0.881,10,-6.119,0,0.0849,0.07220,0.000034,0.3620,0.586,120.017,227347,4
3,Drake,Forever,5UsLjwBaTHBX4ektWIr4XX,71,0.457,0.906,5,-2.278,0,0.3420,0.24900,0.000000,0.1820,0.540,104.020,357707,5
4,Slipknot,Wait and Bleed,2gscB6kDOmrv1P6qs2KXE3,68,0.382,0.996,7,-4.119,1,0.1040,0.00208,0.000000,0.4170,0.327,93.345,147840,4


In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 897
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       898 non-null    object 
 1   track_name        898 non-null    object 
 2   track_id          898 non-null    object 
 3   popularity        898 non-null    int64  
 4   danceability      898 non-null    float64
 5   energy            898 non-null    float64
 6   key               898 non-null    int64  
 7   loudness          898 non-null    float64
 8   mode              898 non-null    int64  
 9   speechiness       898 non-null    float64
 10  acousticness      898 non-null    float64
 11  instrumentalness  898 non-null    float64
 12  liveness          898 non-null    float64
 13  valence           898 non-null    float64
 14  tempo             898 non-null    float64
 15  duration_ms       898 non-null    int64  
 16  time_signature    898 non-null    int64  
dt

In [218]:

df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [219]:
df.to_csv('SpotifyAudioFeatures23.csv')

In [220]:
df.head()

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Kid Cudi,Day 'N' Nite (nightmare),5FEXPoPnzueFJQCPRIrC3c,69,0.871,0.440,11,-6.250,0,0.0761,0.49900,0.000004,0.1140,0.799,137.996,221267,4
1,Jay Sean,Down,6cmm1LMvZdB5zsCwX5BjqE,77,0.727,0.680,2,-4.498,1,0.0286,0.01010,0.000000,0.0821,0.728,132.012,212507,4
2,Kid Cudi,Up Up & Away,1RUTIdTnFs8lHSc0Zr4UJB,64,0.660,0.881,10,-6.119,0,0.0849,0.07220,0.000034,0.3620,0.586,120.017,227347,4
3,Drake,Forever,5UsLjwBaTHBX4ektWIr4XX,71,0.457,0.906,5,-2.278,0,0.3420,0.24900,0.000000,0.1820,0.540,104.020,357707,5
4,Slipknot,Wait and Bleed,2gscB6kDOmrv1P6qs2KXE3,68,0.382,0.996,7,-4.119,1,0.1040,0.00208,0.000000,0.4170,0.327,93.345,147840,4
